# Importamos librerías
* Numpy para el trabajo numérico y de listas
* Pandas para el trabajo con data
* Matplotlib.pyplot para graficar
* Seaborn para importar data y otros gráficos auxiliares

In [29]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Cargamos la data
Para facilitar la carga, usaremos el método [`load_dataset()`](https://seaborn.pydata.org/generated/seaborn.load_dataset.html) de Seaborn para cargar la información de pingüinos y lo mandamos a un [dataframe](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html) de pandas.
```
data = sns.load_dataset('penguins')
```

Podemos visualizar los 5 primeros registros con el método [`head()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.head.html)


In [30]:
# data = sns.load_dataset('penguins')
# data.head()

data = sns.load_dataset('titanic')
data.drop(columns='survived', inplace=True)
data.head()

,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


# Dividir la data en train / test
Usando el método [`sample()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.sample.html) especificamos una fracción de 80% para entrenamiento y lo restante para la evaluación.


```
train=data.sample(frac=0.8,random_state=200)
test=data.drop(train.index).sample(frac=1.0)
```

A continuación, separamos las X de las Y por cada uno de los subconjuntos. Podemos usar el método [`drop()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.drop.html) para separar las X y la [selección por columnas](https://pandas.pydata.org/docs/user_guide/dsintro.html#indexing-selection) para las Y.

El subconjunto de evaluación no los vamos a usar hasta despues de haber entrenado el modelo y ajustado parámetros.


In [31]:
train = data.sample(frac=0.8, random_state=200)
test = data.drop(train.index).sample(frac=1.0, random_state=200)

# target = 'species'
target = 'alive'

y_train = train[target]
x_train = train.drop(columns=target)

y_test = test[target]
x_test = test.drop(columns=target)

# Crear un método para categorizar las columnas según el tipo de dato.

Crea un método que reciba un DataFrame X que contiene las variables independientes de nuestra data y que retorne lo siguiente:
* Lista de columnas con datos binarios
* Lista de columnas con datos categóricos
* Lista de columnas con datos numéricos

Puedes usar el método [`select_dtypes`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.select_dtypes.html) para seleccionar aquellas columnas que contienen `int64` y `float64` para los números y `object` para las categóricas. Adicionalmente, si la columna categórica solo contiene 2 valores únicos, podría considerarse binaria. Para ello podemos usar el método [`nunique()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.nunique.html)

Con tener información sobre las columnas, podemos determinar qué hacer con cada una de ellas. Esta tarea también se puede hacer manualmente y según lo que el análisis previo haya indicado.

In [32]:
def get_columna_por_categoria(X):
    es_binario = X.select_dtypes(include=['object']).apply(pd.Series.nunique) == 2
    es_numerico = X.select_dtypes(include=['int64','float64']).apply(pd.Series.nunique) > 5

    bin_cols = X[es_binario.index[es_binario]].columns
    cat_cols = X[es_binario.index[~es_binario]].columns
    num_cols = X[es_numerico.index[es_numerico]].columns

    return bin_cols, cat_cols, num_cols

# Construyendo el pipeline
Cuando uno construye un modelo en ML, es necesario aplicar las transformaciones de la data dos veces. Una para el dato de entrenamiento, y el otro para la validación.

El [Pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html) en sklearn encadena los pasos de preprocesamiento de tal forma que facilita la ejecución y entender el código.

Adicionalmente, podemos usar el [ColumnTransformer](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html) para seleccionar a qué columnas aplicar las transformaciones. Por esto hemos clasificado las columnas según el tipo de dato.

In [33]:
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression

In [34]:
bin_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encode', OrdinalEncoder()),
])

cat_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encode', OneHotEncoder(sparse=False)),
])

num_pipe = Pipeline([
    ('imputer', KNNImputer(n_neighbors=2)),
    ('scaler', MinMaxScaler()),
])

In [35]:
bin_cols, cat_cols, num_cols = get_columna_por_categoria(x_train)

In [36]:
preprocesador = ColumnTransformer([
    ('bin', bin_pipe, bin_cols),
    ('cat', cat_pipe, cat_cols),
    ('num', num_pipe, num_cols),
])

## Instanciar el modelo de ML que entrenaremos
En esta etapa podríamos crear varios modelos ML

In [37]:
modelo_ml = LogisticRegression()

Concatenar el modelo a la etapa de preprocesamiento usando Pipeline

In [38]:
from sklearn.decomposition import PCA

pipe = Pipeline([
    ('preprocessor', preprocesador),
    ('pca', PCA(n_components=0.999)),
    ('model', modelo_ml)
])

Entrenamos los transformadores de la data en el preprocesamiento y el modelo de ML usando la data de entrenamiento.

In [39]:
from sklearn import set_config

set_config(display='diagram')

In [40]:
pipe.fit(x_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('bin',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('encode',
                                                                   OrdinalEncoder())]),
                                                  Index(['sex'], dtype='object')),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('encode',
                                                                   OneHotEncoder(sparse=False))]),
                                                  Index(['embarked', 'who', 'embark_town'], dtype='object')),
                                                 ('num',
                                                  Pipeline(steps=[('imputer',
                                                                   KNNImputer(n_neighbors=2)),
                                                                  ('scaler',
                                                                   MinMaxScaler())]),
                                                  Index(['age', 'sibsp', 'parch', 'fare'], dtype='object'))])),
                ('pca', PCA(n_components=0.999)),
                ('model', LogisticRegression())])

Obtenemos el puntaje del modelo, tanto de entrenamiento como de la validación.

Al aplicar el score al x_test, la data original para por el proceso de transformación del preprocesamiento y luego la etapa de predicción y luego lo compara con el y_test para obtener una métrica.

Al predecir, realiza los mismos pasos, pero no compara con un *y* de respuesta

In [41]:
print(pipe.score(x_train, y_train))
print(pipe.score(x_test, y_test))

0.82328190743338
0.797752808988764


In [42]:
from sklearn.ensemble import RandomForestClassifier

modelo = RandomForestClassifier()

In [43]:
modelo = Pipeline([
    ('preprocessor', preprocesador),
    ('model', modelo)
])

modelo.fit(x_train, y_train)

print(modelo.score(x_train, y_train))
print(modelo.score(x_test, y_test))

0.9831697054698457
0.7865168539325843


In [44]:
valores = {
    'island': ['Dream'],
    'sex': ['Female'],
    'bill_length_mm': [36.2],
    'bill_depth_mm': [17.3],
    'flipper_length_mm': [187.0],
    'body_mass_g': [3300.0],
}

valores = {
    'pclass': [1],
    'sex': ['female'],
    'age': [22],
    'sibsp': [1],
    'parch': [1],
    'fare': [67.1],
    'embarked': ['C'],
    'class': ['Second'],
    'who': ['man'],
    'adult_male': [False],
    'deck': ['C'],
    'embark_town': ['Cherbourg'],
    'alone': [True],
}

demo = pd.DataFrame.from_dict(valores)

In [48]:
modelo.predict(demo)

array(['yes'], dtype=object)